# Aerial 6G Call Studio (Colab Demo)

This notebook is a fully functional, Colab-friendly companion to the `web_demo` call planner.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NVIDIA/aerial-cuda-accelerated-ran/blob/main/web_demo/Aerial_6G_Call_Studio_Demo.ipynb)


In [ ]:
import json
from pathlib import Path
from urllib.request import urlopen

import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets

plt.style.use('seaborn-v0_8-darkgrid')


In [ ]:
RAW_DATA_URL = "https://raw.githubusercontent.com/NVIDIA/aerial-cuda-accelerated-ran/main/web_demo/data/demo-data.json"
LOCAL_DATA = Path("web_demo/data/demo-data.json")

if LOCAL_DATA.exists():
    demo_data = json.loads(LOCAL_DATA.read_text(encoding="utf-8"))
    source = f"local file: {LOCAL_DATA}"
else:
    with urlopen(RAW_DATA_URL) as response:
        demo_data = json.load(response)
    source = f"remote URL: {RAW_DATA_URL}"

print(f"Loaded demo data from {source}")
print("Profiles:", ", ".join(p["name"] for p in demo_data["perf_profiles"]))


In [ ]:
TRAFFIC_TUNING = {
    "voice": {"latency_bias": -0.8, "prb_scale": 0.7, "reliability_boost": 0.003, "slice": "URLLC"},
    "video": {"latency_bias": 1.9, "prb_scale": 1.5, "reliability_boost": -0.004, "slice": "eMBB"},
    "iot": {"latency_bias": 0.2, "prb_scale": 0.45, "reliability_boost": 0.006, "slice": "mMTC"},
}

PROFILE_MAP = {p["name"]: p for p in demo_data["perf_profiles"]}


def clamp(value, low, high):
    return min(max(value, low), high)


def generate_sessions(profile_name, traffic_type, concurrency):
    profile = PROFILE_MAP[profile_name]
    tuning = TRAFFIC_TUNING[traffic_type]
    density = profile["num_testcases"] / max(profile["groups"], 1)
    base_latency = 2.4 + profile["groups"] * 0.4 + tuning["latency_bias"]
    base_prb = round((14 + density) * tuning["prb_scale"])
    reliability = clamp(0.992 + tuning["reliability_boost"] - profile["groups"] * 0.0007, 0.97, 0.99999)

    sessions = []
    for idx in range(8):
        jitter = ((idx % 3) - 1) * 0.28
        sessions.append(
            {
                "session_id": f"{profile_name.upper()}-{idx + 1:03d}",
                "slice": tuning["slice"],
                "prbs": max(8, round(base_prb + idx * 2)),
                "latency_ms": clamp(base_latency + jitter + concurrency / 140, 1.2, 20),
                "reliability": clamp(reliability - idx * 0.0003, 0.95, 0.99999),
            }
        )

    throughput_gbps = concurrency * base_prb * (0.0016 if traffic_type == "video" else 0.0007)
    gpu_load = clamp((concurrency / 2.2) + profile["groups"] * 2.3, 8, 100)
    avg_latency = sum(s["latency_ms"] for s in sessions) / len(sessions)

    metrics = {
        "throughput_gbps": throughput_gbps,
        "avg_latency_ms": avg_latency,
        "target_reliability": reliability,
        "gpu_load_index": gpu_load,
    }
    return sessions, metrics


In [ ]:
def render_call_plan(profile_name, traffic_type, concurrency):
    sessions, metrics = generate_sessions(profile_name, traffic_type, concurrency)
    df = pd.DataFrame(sessions)

    metric_df = pd.DataFrame(
        {
            "Metric": ["Estimated Throughput", "Avg Latency", "Target Reliability", "GPU Load Index"],
            "Value": [
                f"{metrics['throughput_gbps']:.2f} Gbps",
                f"{metrics['avg_latency_ms']:.2f} ms",
                f"{metrics['target_reliability'] * 100:.3f}%",
                f"{metrics['gpu_load_index']:.1f} / 100",
            ],
        }
    )

    display(metric_df)
    display(df.style.format({"latency_ms": "{:.2f}", "reliability": "{:.5f}"}))

    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    ax[0].bar(df["session_id"], df["latency_ms"], color="#4F81BD")
    ax[0].set_title("Session Latency")
    ax[0].set_ylabel("ms")
    ax[0].tick_params(axis="x", rotation=45)

    ax[1].plot(df["session_id"], df["prbs"], marker="o", color="#9BBB59")
    ax[1].set_title("PRB Allocation")
    ax[1].set_ylabel("PRBs")
    ax[1].tick_params(axis="x", rotation=45)

    plt.tight_layout()
    plt.show()


profile_widget = widgets.Dropdown(options=sorted(PROFILE_MAP.keys()), description="Profile")
traffic_widget = widgets.Dropdown(options=["voice", "video", "iot"], value="video", description="Traffic")
concurrency_widget = widgets.IntSlider(value=80, min=10, max=200, step=5, description="Concurrency")

ui = widgets.VBox([profile_widget, traffic_widget, concurrency_widget])
out = widgets.interactive_output(
    render_call_plan,
    {
        "profile_name": profile_widget,
        "traffic_type": traffic_widget,
        "concurrency": concurrency_widget,
    },
)

display(ui, out)


## Notes

- This notebook reproduces the same synthetic KPI/session logic used in `web_demo/main.js`.
- Use it for quick demos, experimentation, or as a baseline before connecting real runtime telemetry.
